# Gerar 10 folds para validação cruzada do conjunto de dados do CohQuAD In en.

Divide o conjunto de dados em 10 folds.

Cada fold possui 90% para dados de treinamento e 10% para dados de teste.

Os dados de teste não se repetem em cada fold.

O procedimento exclui os documentos que geram mais de 512 tokens.

Utiliza os arquivos para gerar K folds:
- `original.zip`
- `originalpos.zip`
- `perturbado_pX_kY.zip`
- `perturbadopos_pX_kY.zip`

Nos nomes dos arquivos `perturbado_pX_kY.zip`,`perturbadopos_pX_kY.zip`, X é o número de documentos perturbados e Y o valor de top K predições.

Cria o arquivo `COHEBERT_KFOLD_10_PX_KY.zip`, X é o número de documentos perturbados e Y o valor de top K predições.

----------------------------

**Link biblioteca Transformers:**
https://github.com/huggingface/transformers


**Artigo original BERT:**
https://arxiv.org/pdf/1506.06724.pdf

# 1 Preparação do ambiente
Preparação do ambiente para execução do notebook.

## 1.1 Tempo inicial de processamento

In [1]:
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

In [2]:
print("  Tempo de início de processamento:  {:} (h:mm:ss)".format(inicio_processamento))

  Tempo de início de processamento:  1722360823.2017791 (h:mm:ss)


## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [3]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.
    """

    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))

    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [4]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.

      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.
    """

    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)

    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download
    nome_arquivo = nome_arquivo_destino.split("/")[-1]

    # Define o nome e caminho do arquivo temporário
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"

    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))

    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:
        tamanho_conteudo = data.headers.get("Content-Length")
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):
            if chunk:
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)

    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)

    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [5]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """

    import re

    documentoLimpo = re.compile("<.*?>")
    return re.sub(documentoLimpo, "", documento)

Funções auxiliares de arquivos

In [6]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "

    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [7]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)

    sentencas = []
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())

    # Fecha o arquivo
    arquivo.close()

    return sentencas

In [8]:
def salvar(nome_arquivo,texto):
    """
      Salva um texto em arquivo.

      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [9]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Calcula a média de uma lista tempo string no formato hh:mm:ss.

In [10]:
# Import das bibliotecas.
from cmath import rect, phase
from math import radians, degrees

def mediaAngulo(deg):
    return degrees(phase(sum(rect(1, radians(d)) for d in deg)/len(deg)))

def mediaTempo(tempos):
    '''
    Calcula a média de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Verifica se deu algum dia
    dia = 24 * 60 * 60
    # Converte para angulos
    para_angulos = [s * 360. / dia for s in segundos]
    # Calcula a média dos angulos
    media_como_angulo = mediaAngulo(para_angulos)
    media_segundos = media_como_angulo * dia / 360.
    if media_segundos < 0:
        media_segundos += dia
    # Recupera as horas e os minutos
    h, m = divmod(media_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Calcula a soma de uma lista de tempo string no formato hh:mm:ss

In [11]:
def somaTempo(tempos):
    '''
    Calcula a soma de uma lista de tempo string no formato hh:mm:ss
    '''
    t = (tempo.split(':') for tempo in tempos)
    # Converte para segundos
    segundos = ((float(s) + int(m) * 60 + int(h) * 3600) for h, m, s in t)
    # Soma os segundos
    soma_segundos = sum([s * 1. for s in segundos])
    # Recupera as horas e os minutos
    h, m = divmod(soma_segundos, 3600)
    # Recupera os minutos e os segundos
    m, s = divmod(m, 60)
    return '{:02d}:{:02d}:{:02d}'.format(int(h), int(m), int(s))

Classe(ModeloArgumentosMedida) de definição dos parâmetros do modelo para medida

In [12]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )

Classe(ModeloArgumentosClassificacao) de definição dos parâmetros do modelo para classificação

In [13]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosClassificacao:
    '''
    Classe(ModeloArgumentosClassificacao) de definição dos parâmetros do modelo BERT para a classificação de coerência.
    '''
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )
    num_labels: int = field(
        default=2,
        metadata={"help": "número de rótulos a serem classificados."},
    )
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    optimizer: str = field(
        default="AdamW",
        metadata={"help": "otimizador do modelo."},
    )
    use_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o uso do wandb."},
    )
    salvar_modelo_wandb : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do modelo no wandb."},
    )
    salvar_modelo : bool = field(
        default=False,
        metadata={"help": "habilita o salvamento do modelo."},
    )
    salvar_avaliacao : bool = field(
        default=True,
        metadata={"help": "habilita o salvamento do resultado da avaliação."},
    )
    salvar_classificacao : bool = field(
        default=False,
        metadata={"help": "habilita o salvamento da classificação."},
    )
    usar_mcl_ajustado: bool = field(
        default=False,
        metadata={'help': 'habilita o carragamento de mcl ajustado.'},
    )
    documentos_perturbados: int = field(
        default="1",
        metadata={"help": "Quantidade de documentos perturbados comparados com o seu original."},
    )
    top_k_predicao: int = field(
        default="100",
        metadata={"help": "Quantidade de previsões de palavras recuperadas mais próximas da máscara."},
    )
    fold: int = field(
        default="1",
        metadata={"help": "fold a ser avaliado."},
    )

Biblioteca de limpeza de tela

In [14]:
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [15]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [16]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [17]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [18]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def getDeviceGPU():
    """
    Retorna um dispositivo de GPU se disponível ou CPU.

    Retorno:
    `device` - Um device de GPU ou CPU.
    """

    # Se existe GPU disponível.
    if torch.cuda.is_available():

        # Diz ao PyTorch para usar GPU.
        device = torch.device("cuda")

        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")

    return device

In [19]:
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Conecta o modelo ao device

In [20]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def conectaGPU(model, device):
    """
      Conecta um modelo BERT a GPU.

      Parâmetros:
        `model` - Um modelo BERT carregado.
        `device` - Um device de GPU.

      Retorno:
        `model` - Um objeto model BERT conectado a GPU.
    """
    # Associa a GPU ao modelo.
    model.to(device)

    # Se existe GPU disponível.
    if torch.cuda.is_available():
        # Diga ao pytorch para rodar este modelo na GPU.
        logging.info("Pytorch rodando o modelo na GPU.")
        model.cuda()

    else:
        logging.info("Pytorch rodando sem GPU.")

    return model

Memória

Memória disponível no ambiente

In [21]:
# Import das bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  16.8 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.



In [22]:
# # Monta o Google Drive para esta instância de notebook.
# from google.colab import drive

# drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()

from google.cloud import storage

# Initialize a client
client = storage.Client(project='mestrado-424014')

bucket_name = 'mestrado_rita_01'
bucket = client.get_bucket(bucket_name)

# List files in the bucket
blobs = bucket.list_blobs(prefix="cohquad/")
for blob in blobs:
    print(blob.name)


# Função para listar blobs dentro de uma pasta específica
def list_blobs_with_prefix(bucket_name, prefix):
    """Lists all the blobs in the bucket that begin with the prefix."""
    blobs = bucket.list_blobs(prefix=prefix)
    for blob in blobs:
        print(blob.name)

# Exemplo de uso: listar arquivos na pasta 'cohquad/'
list_blobs_with_prefix(bucket_name, 'cohquad/')


cohquad/
cohquad/msrp_test.csv
cohquad/msrp_test.xlsx
cohquad/msrp_train.csv
cohquad/msrp_train.xlsx
cohquad/original.zip
cohquad/originalner.zip
cohquad/originalpos.zip
cohquad/
cohquad/msrp_test.csv
cohquad/msrp_test.xlsx
cohquad/msrp_train.csv
cohquad/msrp_train.xlsx
cohquad/original.zip
cohquad/originalner.zip
cohquad/originalpos.zip


In [23]:
# bucket = client.get_bucket('mestrado_rita_01')

# # List files in the bucket
# blobs = bucket.list_blobs()
# for blob in blobs:
#     print(blob.name)

# # Download a file
# # Correct usage: just provide the file name within the bucket
# blob = bucket.blob('msrp_test.xlsx')  # Only the file path relative to the bucket root

# # Now download the file
# blob.download_to_filename('msrp_test.xlsx')



## 1.7 Instalação do wandb

Instalação

In [24]:
!pip install --upgrade wandb

Login via linha de comando

In [26]:
!wandb login a55ff08814d3b18b89f2e4938ab69b136c33b2e1

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.8 Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face.

In [ ]:
!pip install -U transformers==4.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


# 2 Parametrização

## Gerais

In [ ]:
# Definição dos parâmetros a serem avaliados
#Quantidade de documentos a serem perturbados a partir do original.
DOCUMENTOS_PERTURBADOS = 1

#Quantidade de palavras a serem recuperadas mais próximas da máscara.
TOP_K_PREDICAO = 1

## Específicos

Parâmetros do modelo

In [ ]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosClassificacao(
    max_seq_len = 512,
    pretrained_model_name_or_path = "bert-large-cased",
    #pretrained_model_name_or_path = "bert-base-cased"
    #pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",

    do_lower_case = False, # default True
    num_labels = 2,
    output_attentions = False, # default False
    output_hidden_states = False, # default False
    optimizer = 'AdamW',
    use_wandb = False, # Ativa a gravação de logs no wandb
    salvar_modelo_wandb = False, # Ativa o salvamento do MCL no wandb
    salvar_modelo = False, # Ativa o salvamento do MCL no googledrive
    salvar_avaliacao = False, # Salva o resultado classificações
    salvar_classificacao = False, # Salva o resultado da avaliação das classificações
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path.
    documentos_perturbados = DOCUMENTOS_PERTURBADOS, # Quantidade de documentos a serem perturbados a partir do original.
    top_k_predicao = TOP_K_PREDICAO, # Conjunto de valores: 1, 10, 100, 500 e 1000. Quantidade de palavras a serem recuperadas mais próximas da máscara.
    fold = 10, # Quantidade de folds a serem gerados.
)

## Nome do diretório dos arquivos de dados

In [ ]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "cohquad"

## Define o caminho para os arquivos de dados

In [ ]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

## Inicialização diretórios

Diretório base local

In [ ]:
# Importando as bibliotecas.
import os

def criaDiretorioLocal():

  # Cria o diretório para receber os arquivos Originais e Permutados
  # Diretório a ser criado
  dirbase = DIRETORIO_LOCAL[:-1]

  if not os.path.exists(dirbase):
      # Cria o diretório
      os.makedirs(dirbase)
      logging.info("Diretório criado: {}.".format(dirbase))
  else:
      logging.info("Diretório já existe: {}.".format(dirbase))

In [ ]:
criaDiretorioLocal()

INFO:root:Diretório criado: /content/cohquad.


Diretório para conter as os resultados das classificações

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacao():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "/validacao_classificacao_palavra"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacao()

INFO:root:Diretório criado: /content/drive/MyDrive/Colab Notebooks/Data/cohquad//validacao_classificacao_palavra.


Diretório para conter os arquivos da avaliação kfold

In [ ]:
# Import de bibliotecas.
import os

def criaDiretorioClassificacaoKfold():
  DIRETORIO_BASE = DIRETORIO_DRIVE + "validacao_classificacao_palavra/kfold"

  # Verifica se o diretório existe
  if not os.path.exists(DIRETORIO_BASE):
    # Cria o diretório
    os.makedirs(DIRETORIO_BASE)
    logging.info("Diretório criado: {}.".format(DIRETORIO_BASE))
  else:
    logging.info("Diretório já existe: {}.".format(DIRETORIO_BASE))

In [ ]:
criaDiretorioClassificacaoKfold()

INFO:root:Diretório criado: /content/drive/MyDrive/Colab Notebooks/Data/cohquad/validacao_classificacao_palavra/kfold.


# 3 BERT

## 3.1 Modelo Pré-treinado BERT

### Funções Auxiliares

In [ ]:
def getNomeModeloBERT(model_args):
    '''
    Recupera uma string com uma descrição do modelo BERT para nomes de arquivos e diretórios.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `MODELO_BERT` - Nome do modelo BERT.
    '''

    # Verifica o nome do modelo(default SEM_MODELO_BERT)
    MODELO_BERT = "SEM_MODELO_BERT"

    if 'neuralmind' in model_args.pretrained_model_name_or_path:
        MODELO_BERT = "_BERTimbau"

    else:
        if 'multilingual' in model_args.pretrained_model_name_or_path:
            MODELO_BERT = "_BERTmultilingual"

    return MODELO_BERT

In [ ]:
def getTamanhoBERT(model_args):
    '''
    Recupera uma string com o tamanho(dimensão) do modelo BERT para nomes de arquivos e diretórios.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `TAMANHO_BERT` - Nome do tamanho do modelo BERT.
    '''

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = "_large"

    if 'base' in model_args.pretrained_model_name_or_path:
        TAMANHO_BERT = "_base"

    return TAMANHO_BERT

### Função download Modelo Pre-treinado BERT

In [ ]:
# Import das bibliotecas.
import zipfile # Biblioteca para descompactar
import shutil # iblioteca de manipulação arquivos de alto nível

def downloadModeloPretreinado(model_args):
    """
      Realiza o download do modelo BERT(MODELO) e retorna o diretório onde o modelo BERT(MODELO) foi descompactado.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `DIRETORIO_MODELO` - Diretório de download do modelo.
    """

    # Nome diretório base modelo BERT
    NOME_DIRETORIO_BASE_MODELO = "modeloBERT"

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Recupera o nome ou caminho do modelo
    MODELO = model_args.pretrained_model_name_or_path

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in MODELO:
        URL_MODELO = MODELO

    # Se a variável foi setada.
    if URL_MODELO:

        # Diretório do modelo.
        DIRETORIO_MODELO = DIRETORIO_COHEBERT + "/" + NOME_DIRETORIO_BASE_MODELO

        # Recupera o nome do arquivo do modelo da url.
        NOME_ARQUIVO = URL_MODELO.split("/")[-1]

        # Nome do arquivo do vocabulário.
        ARQUIVO_VOCAB = "vocab.txt"

        # Caminho do arquivo na url.
        CAMINHO_ARQUIVO = URL_MODELO[0:len(URL_MODELO)-len(NOME_ARQUIVO)]

        # Verifica se o diretório de descompactação existe no diretório corrente
        if os.path.exists(DIRETORIO_MODELO):
            logging.info("Apagando diretório existente do modelo!")
            # Apaga o diretório e os arquivos existentes
            shutil.rmtree(DIRETORIO_MODELO)

        # Realiza o download do arquivo do modelo
        downloadArquivo(URL_MODELO, NOME_ARQUIVO)

        # Descompacta o arquivo no diretório de descompactação.
        arquivo_zip = zipfile.ZipFile(NOME_ARQUIVO, "r")
        arquivo_zip.extractall(DIRETORIO_MODELO)

        # Baixa o arquivo do vocabulário.
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente.
        URL_MODELO_VOCAB = CAMINHO_ARQUIVO + ARQUIVO_VOCAB
        # Coloca o arquivo do vocabulário no diretório do modelo.
        downloadArquivo(URL_MODELO_VOCAB, DIRETORIO_MODELO + "/" + ARQUIVO_VOCAB)

        # Apaga o arquivo compactado
        os.remove(NOME_ARQUIVO)

        logging.info("Diretório {} do modelo BERT pronta!".format(DIRETORIO_MODELO))

    else:
        DIRETORIO_MODELO = MODELO
        logging.info("Variável URL_MODELO não setada!")

    return DIRETORIO_MODELO

### Copia o modelo do BERT ajustado

In [ ]:
# Import das bibliotecas.
import shutil # iblioteca de manipulação arquivos de alto nível

def copiaModeloAjustado(model_args):
    """
      Copia o modelo ajustado BERT do GoogleDrive para o projeto.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `DIRETORIO_LOCAL_MODELO_AJUSTADO` - Diretório de download ajustado do modelo.
    """

    # Verifica o nome do modelo BERT a ser utilizado
    MODELO_BERT = getNomeModeloBERT(model_args)

    # Verifica o tamanho do modelo(default large)
    TAMANHO_BERT = getTamanhoBERT(model_args)

    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()

    # Diretório local de salvamento do modelo.
    DIRETORIO_LOCAL_MODELO_AJUSTADO = DIRETORIO_COHEBERT + "/modelo_ajustado/"

    # Diretório remoto de salvamento do modelo no google drive.
    DIRETORIO_REMOTO_MODELO_AJUSTADO = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/validacao_classificacao_palavra/holdout/modelo/" + MODELO_BERT + TAMANHO_BERT

    # Copia o arquivo do modelo para o diretório no Google Drive.
    shutil.copytree(DIRETORIO_REMOTO_MODELO_AJUSTADO, DIRETORIO_LOCAL_MODELO_AJUSTADO)

    logging.info("Modelo BERT ajustado copiado!")

    return DIRETORIO_LOCAL_MODELO_AJUSTADO

### Verifica de onde utilizar o modelo do BERT

In [ ]:
def verificaModelo(model_args):
    """
    Verifica de onde utilizar o modelo.

    Parâmetros:
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `DIRETORIO_MODELO` - Diretório de download do modelo.
    """

    DIRETORIO_MODELO = None

    if model_args.usar_mcl_ajustado == True:
        # Diretório do modelo
        DIRETORIO_MODELO = copiaModeloAjustado()

        logging.info("Usando modelo BERT ajustado.")

    else:
        DIRETORIO_MODELO = downloadModeloPretreinado(model_args)
        logging.info("Usando modelo BERT pré-treinado.")

    return DIRETORIO_MODELO

## 3.2 Tokenizador BERT

### Função carrega Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).



In [ ]:
# Import das bibliotecas.
from transformers import BertTokenizer # Importando as bibliotecas do tokenizador BERT.

def carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args):
    """
      Carrega o tokenizador do DIRETORIO_MODELO.
      O tokenizador utiliza WordPiece.
      Carregando o tokenizador do diretório "./modelo/" do diretório padrão se variável `DIRETORIO_MODELO` setada.
      Caso contrário carrega da comunidade
      Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.
      O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado a partir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `tokenizer` - Tokenizador BERT.
    """

    tokenizer = None

    # Se a variável DIRETORIO_MODELO foi setada.
    if DIRETORIO_MODELO:
        # Carregando o Tokenizador.
        logging.info("Carregando o tokenizador BERT do diretório {}.".format(DIRETORIO_MODELO))

        tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, do_lower_case=model_args.do_lower_case)

    else:
        # Carregando o Tokenizador da comunidade.
        logging.info("Carregando o tokenizador BERT da comunidade.")

        tokenizer = BertTokenizer.from_pretrained(model_args.pretrained_model_name_or_path, do_lower_case=model_args.do_lower_case)

    return tokenizer

## 3.3 Carrega o modelo e tokenizador BERT

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **"neuralmind/bert-base-portuguese-cased"**
* **"neuralmind/bert-large-portuguese-cased"**

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

### Função carrega modelo BERT medida




In [ ]:
# Import das bibliotecas.
from transformers import BertModel # Importando as bibliotecas do Modelo BERT.

def carregaModeloMedida(DIRETORIO_MODELO, model_args):
    """
      Carrega o modelo e retorna o modelo.

      Parâmetros:
        `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.
        `model_args` - Objeto com os argumentos do modelo.

      Retorno:
        `model` - Um objeto do modelo BERT carregado.
    """

    # Variável para setar o arquivo.
    URL_MODELO = None

    if "http" in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para cálculo de medidas.".format(DIRETORIO_MODELO))

        model = BertModel.from_pretrained(DIRETORIO_MODELO,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para cálculo de medidas.".format(model_args.pretrained_model_name_or_path))

        model = BertModel.from_pretrained(model_args.pretrained_model_name_or_path,
                                          output_attentions=model_args.output_attentions,
                                          output_hidden_states=model_args.output_hidden_states)

    return model

### Função carrega modelo BERT classificação



In [ ]:
# Import das bibliotecas.
from transformers import BertForSequenceClassification # Importando as bibliotecas do Modelo BERT.

def carregaModeloClassifica(DIRETORIO_MODELO, model_args):
    '''
    Carrega o modelo e retorna o modelo.

    Parâmetros:
    `DIRETORIO_MODELO` - Diretório a ser utilizado pelo modelo BERT.
    `model_args` - Objeto com os argumentos do modelo.

    Retorno:
    `model` - Um objeto do modelo BERT carregado.
    '''

    # Variável para setar o arquivo.
    URL_MODELO = None

    if 'http' in model_args.pretrained_model_name_or_path:
        URL_MODELO = model_args.pretrained_model_name_or_path

    # Se a variável URL_MODELO foi setada
    if URL_MODELO:
        # Carregando o Modelo BERT
        logging.info("Carregando o modelo BERT do diretório {} para classificação.".format(DIRETORIO_MODELO))

        model = BertForSequenceClassification.from_pretrained(DIRETORIO_MODELO,
                                                              num_labels=model_args.num_labels,
                                                              output_attentions=model_args.output_attentions,
                                                              output_hidden_states=model_args.output_hidden_states)

    else:
        # Carregando o Modelo BERT da comunidade
        logging.info("Carregando o modelo BERT da comunidade {} para classificação.".format(model_args.pretrained_model_name_or_path))

        model = BertForSequenceClassification.from_pretrained(model_args.pretrained_model_name_or_path,
                                                              num_labels=model_args.num_labels,
                                                              output_attentions=model_args.output_attentions,
                                                              output_hidden_states=model_args.output_hidden_states)
    return model

### Função carrega o BERT

In [ ]:
def carregaBERT(model_args):
    """
      Carrega o BERT para cálculo de medida ou classificação e retorna o modelo e o tokenizador.
      O tipo do model retornado pode ser BertModel ou BertForSequenceClassification, depende do tipo de model_args.

      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.
          - Se model_args = ModeloArgumentosClassificacao deve ser carregado o BERT para classificação(BertForSequenceClassification).
          - Se model_args = ModeloArgumentosMedida deve ser carregado o BERT para cálculo de medida(BertModel).

      Retorno:
        `model` - Um objeto do modelo BERT carregado.
        `tokenizer` - Um objeto tokenizador BERT carregado.
    """

    # Verifica a origem do modelo
    DIRETORIO_MODELO = verificaModelo(model_args)

    # Variável para conter o modelo
    model = None

    # Verifica o tipo do modelo em model_args
    if type(model_args) == ModeloArgumentosMedida:
        # Carrega o modelo para cálculo da medida
        model = carregaModeloMedida(DIRETORIO_MODELO, model_args)

    else:
        # Carrega o modelo para classificação
        model = carregaModeloClassifica(DIRETORIO_MODELO, model_args)

        # Recupera o dispotivo da GPU
        device = getDeviceGPU()

        # Conecta o modelo a GPU
        model = conectaGPU(model, device)

    # Carrega o tokenizador.
    # O tokenizador é o mesmo para o classificador e medidor.
    tokenizer = carregaTokenizadorModeloPretreinado(DIRETORIO_MODELO, model_args)

    return model, tokenizer

### Carrega o BERT

In [ ]:
# Carrega o modelo e tokenizador do BERT
model, tokenizer = carregaBERT(model_args)

INFO:root:Variável URL_MODELO não setada!
INFO:root:Usando modelo BERT pré-treinado.
INFO:root:Carregando o modelo BERT da comunidade bert-large-cased para classificação.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:root:Sem GPU disponível, usando CPU.
INFO:root:Pytorch rodando sem GPU.
INFO:root:Carregando o tokenizador BERT do diretório bert-large-cased.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

### Recupera detalhes do BERT

In [ ]:
# Verifica o nome do modelo BERT a ser utilizado
MODELO_BERT = getNomeModeloBERT(model_args)

# Verifica o tamanho do modelo(default large)
TAMANHO_BERT = getTamanhoBERT(model_args)

# 4 Divisão dos Folds

## 4.1 Carregamento dos arquivos de dados originais

### 4.1.1 Especifica os nomes dos arquivos de dados



In [ ]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "dataset.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"
NOME_ARQUIVO_ORIGINAL_POS = "datasetpos.csv"
NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO = "originalpos.zip"

NOME_ARQUIVO_TREINAMENTO = "msrp_train.csv"
NOME_ARQUIVO_TESTE = "msrp_test.csv"

### 4.1.2 Cria o diretório local para receber os dados

In [ ]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}.".format(dirbase))
else:
    logging.info("Diretório já existe: {}.".format(dirbase))

INFO:root:Diretório já existe: /content/cohquad.


### 4.1.3 Copia os arquivos do Google Drive para o Colaboratory

In [ ]:
# # Se estiver executando no Google Colaboratory
# if IN_COLAB:

#   !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL"
#   !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" "$DIRETORIO_LOCAL"

#   !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_TREINAMENTO" "$DIRETORIO_LOCAL"
#   !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_TESTE" "$DIRETORIO_LOCAL"

#   logging.info("Terminei a cópia.")

  # Function to download a file from GCS

# Função para baixar um blob
def download_blob(bucket_name, source_blob_name, destination_file_name):
    """Downloads a blob from the bucket."""
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)
    print(f"Blob {source_blob_name} downloaded to {destination_file_name}.")

    # Função para fazer upload de um arquivo
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")


# Exemplo de uso: baixar arquivos da pasta 'cohquad/'
arquivos_para_baixar = [
    'cohquad/msrp_test.xlsx',
    'cohquad/msrp_train.xlsx',
    'cohquad/original.zip',
    'cohquad/originalner.zip',
    'cohquad/originalpos.zip'
]

for arquivo in arquivos_para_baixar:
    download_blob(bucket_name, arquivo, arquivo)  # Salva com o mesmo nome no diretório local



Blob cohquad/msrp_test.xlsx downloaded to cohquad/msrp_test.xlsx.
Blob cohquad/msrp_train.xlsx downloaded to cohquad/msrp_train.xlsx.
Blob cohquad/original.zip downloaded to cohquad/original.zip.
Blob cohquad/originalner.zip downloaded to cohquad/originalner.zip.
Blob cohquad/originalpos.zip downloaded to cohquad/originalpos.zip.


Descompacta os arquivos

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens
*   `-d` Diretório de destino


In [ ]:

# # Se estiver executando no Google Colaboratory
# if IN_COLAB:
#   !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" -d "$DIRETORIO_LOCAL"
#   !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

#   logging.info("Terminei a descompactação.")

import zipfile

files_to_unzip = [
    "original.zip",
    "originalner.zip",
    "originalpos.zip",
    # Adicione outros arquivos conforme necessário
]

for file in files_to_unzip:
    with zipfile.ZipFile(os.path.join(DIRETORIO_LOCAL, file), 'r') as zip_ref:
        zip_ref.extractall(DIRETORIO_LOCAL)

logging.info("Terminei a descompactação.")

INFO:root:Terminei a descompactação.


### 4.1.4 Carregamento das lista com os dados dos arquivos originais

#### Carrega o arquivo dos dados originais e POS

In [ ]:
# Importando as bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")
lista_documentos_originais_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
logging.info("TERMINADO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

df_train = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_TREINAMENTO, sep=",", encoding="UTF-8")
df_test = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_TESTE, sep=",", encoding="UTF-8")

logging.info("TERMINADO TREINO: {}.".format(len(df_train)))
logging.info("TERMINADO TESTE: {}.".format(len(df_test)))

INFO:root:TERMINADO ORIGINAIS: 10948.
INFO:root:TERMINADO ORIGINAIS POS: 10948.
INFO:root:TERMINADO TREINO: 3929.
INFO:root:TERMINADO TESTE: 1612.


In [ ]:
lista_documentos_originais.sample(5)

,id,sentencas,documento
5048,1438101,"['In late morning trading, the Dow was up 13.8...","In late morning trading, the Dow was up 13.88,..."
10758,375021,"['Davey started attending Northwest College, w...","Davey started attending Northwest College, whi..."
10130,619375,"['Ultimately, she will earn more than the $900...","Ultimately, she will earn more than the $900,0..."
6669,121626,['District Attorney Dave Lupas reminded jurors...,District Attorney Dave Lupas reminded jurors t...
9812,1380520,"['Thousands of revellers, celebrating the deci...","Thousands of revellers, celebrating the decisi..."


In [ ]:
lista_documentos_originais_pos.sample(5)

,id,pos_documento
8676,692176,"[[['Heavy', 'weekend', 'rain', 'and', 'runoff'..."
7922,2795258,"[[['These', 'documents', 'are', 'indecipherabl..."
8055,2904808,"[[['By', 'Sunday', 'night', ',', 'the', 'fires..."
4628,1708719,"[[['They', 'said', 'it', 'affects', 'all', 'de..."
1331,1703383,"[[['Hoffa', ',', '62', ',', 'vanished', 'on', ..."


In [ ]:
df_train.sample(5)

,Quality,#1 ID,#2 ID,#1 String,#2 String
1371,1,2553320,2553127,"At first blush, then, the distinction drawn by...","At first blush, then, the creators of the regi..."
2768,0,1892983,1892817,BA shares were up 0.59 percent at 170 pence in...,BA shares closed down 0.89 percent at 167-1/2 ...
849,1,2635090,2635234,Experts said the case marks one of the first t...,Legal experts say the case may mark the first ...
2538,0,368021,368077,PowderJect is roughly 20 percent owned by CEO ...,"PowderJect, roughly 20 percent owned by Chief ..."
212,0,1707092,1707031,The company's operating loss rose 59 percent t...,Operating revenue fell 4.5 percent to $2.3 bil...


In [ ]:
df_test.sample(5)

,Quality,#1 ID,#2 ID,#1 String,#2 String
94,1,1378004,1378107,The upcoming second-quarter earnings season wi...,They say second-quarter earnings reports will ...
1253,1,316726,316927,"But they did not, as of the time of this parti...",They did not ... provide the security that we ...
616,1,2877244,2877197,A coalition of states petitioned a federal app...,New Jersey was one of eleven states that asked...
1092,1,1661193,1661317,Close co-operation between law-enforcement age...,Close cooperation between regional law enforce...
1418,1,511204,510624,"Among those waiting a turn was Jodie Singer, a...","Jodie Singer, a sixth-grader from Washington, ..."


#### Corrigir os tipos de colunas dos dados originais e POS

Em dados originais:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em dados originais pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [ ]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

def corrigirTipoDadosColunasOriginais(lista_documentos_originais, lista_documentos_originais_pos):

  # Verifica se o tipo da coluna não é list e converte
  lista_documentos_originais["sentencas"] = lista_documentos_originais["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

  lista_documentos_originais_pos["pos_documento"] = lista_documentos_originais_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

  logging.info("TERMINADO CORREÇÃO ORIGINAIS: {}.".format(len(lista_documentos_originais)))
  logging.info("TERMINADO CORREÇÃO ORIGINAIS POS: {}.".format(len(lista_documentos_originais_pos)))

  return lista_documentos_originais, lista_documentos_originais_pos

In [ ]:
lista_documentos_originais, lista_documentos_originais_pos = corrigirTipoDadosColunasOriginais(lista_documentos_originais, lista_documentos_originais_pos)

INFO:root:TERMINADO CORREÇÃO ORIGINAIS: 10948.
INFO:root:TERMINADO CORREÇÃO ORIGINAIS POS: 10948.


#### Criando dados indexados originais

In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_originais_indexado = lista_documentos_originais.set_index(["id"])
lista_documentos_originais_indexado.head()

,sentencas,documento
id,,
702876,"[Amrozi accused his brother, whom he called th...","Amrozi accused his brother, whom he called the..."
702977,"[Referring to him as only the witness"", Amrozi...","Referring to him as only the witness"", Amrozi ..."
2108705,[Yucaipa owned Dominick's before selling the c...,Yucaipa owned Dominick's before selling the ch...
2108831,[Yucaipa bought Dominick's in 1995 for $693 mi...,Yucaipa bought Dominick's in 1995 for $693 mil...
1330381,[They had published an advertisement on the In...,They had published an advertisement on the Int...


In [ ]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
lista_documentos_originais_pos_indexado = lista_documentos_originais_pos.set_index(["id"])
lista_documentos_originais_pos_indexado.head()

,pos_documento
id,
702876,"[[[Amrozi, accused, his, brother, ,, whom, he,..."
702977,"[[[Referring, to, him, as, only, the, witness,..."
2108705,"[[[Yucaipa, owned, Dominick, 's, before, selli..."
2108831,"[[[Yucaipa, bought, Dominick, 's, in, 1995, fo..."
1330381,"[[[They, had, published, an, advertisement, on..."


In [ ]:

def concatenar_sentencas_e_agrupar(lista_documentos_originais, lista_documentos_originais_pos, df_train, df_test):
    # Primeiro, criaremos um dicionário para mapear os IDs aos pós-tagging
    pos_tagging_map = {row['id']: row['pos_documento'] for index, row in lista_documentos_originais_pos.iterrows()}

    # Criaremos uma lista para armazenar os dados agrupados e concatenados
    dados_concatenados = []

    # Agora, iremos iterar pelos dados de treino e teste para criar os pares de sentenças
    for df in [df_train, df_test]:
        for index, row in tqdm_notebook(df.iterrows(), desc='Concatenando e agrupando dados', leave=False):
            # Concatenação das sentenças com um separador " [SEP] "
            documento_concatenado = row['#1 String'] + " [SEP] " + row['#2 String']

            # Adicionamos a linha formatada no novo esquema ao nosso conjunto de dados concatenados e agrupados
            dados_concatenados.append({
                "id": row['#1 ID'], # Mantém o ID da primeira sentença como identificador único
                "documento": documento_concatenado, # Sentenças concatenadas
                "classe": row['Quality'] # Classe de similaridade
            })

    # Convertendo a lista de dados concatenados para um DataFrame pandas
    df_dados_concatenados = pd.DataFrame(dados_concatenados)
    return df_dados_concatenados

In [ ]:
# Agora vamos chamar a função e passar os datasets carregados
df_dados_concatenados = concatenar_sentencas_e_agrupar(
    lista_documentos_originais,
    lista_documentos_originais_pos,
    df_train,
    df_test
)

# Vamos exibir as primeiras linhas do DataFrame para verificar
print(df_dados_concatenados.head())

Concatenando e agrupando dados: 0it [00:00, ?it/s]

Concatenando e agrupando dados: 0it [00:00, ?it/s]

        id                                          documento  classe
0   702876  Amrozi accused his brother, whom he called "th...       1
1  2108705  Yucaipa owned Dominick's before selling the ch...       0
2  1330381  They had published an advertisement on the Int...       1
3  3344667  Around 0335 GMT, Tab shares were up 19 cents, ...       0
4  1236820  The stock rose $2.11, or about 11 percent, to ...       1


### 4.1.5 Agrupar os dados originais e teste

In [ ]:
from tqdm.notebook import tqdm
import pandas as pd

def agrupar_dados_com_pos_tagging(lista_documentos_originais, lista_documentos_pos, df_train, df_test):
    # Mapeia cada ID de documento ao seu respectivo texto e vetor POS
    # Note que estamos assumindo que lista_documentos_originais e lista_documentos_pos
    # já são DataFrames com as colunas corretamente nomeadas.
    documentos_map = {doc.id: (doc.documento, pos.pos_documento)
                      for doc, pos in zip(lista_documentos_originais.itertuples(), lista_documentos_pos.itertuples())}

    dados_agrupados = []

    # Processa os dataframes de treino e teste
    for df in [df_train, df_test]:
        for index, row in df.iterrows():
            # Concatena as sentenças e os vetores POS tagging com um separador especial
            doc_concat = f"{documentos_map[row['#1 ID']][0]} [SEP] {documentos_map[row['#2 ID']][0]}"
            pos_concat = f"{documentos_map[row['#1 ID']][1]} [SEP] {documentos_map[row['#2 ID']][1]}"

            dados_agrupados.append({
                "id": f"{row['#1 ID']}_{row['#2 ID']}",
                "documento": doc_concat,
                "pos_tagging": pos_concat,
                "classe": row['Quality']
            })

    # Convertendo a lista de dados agrupados para um DataFrame pandas
    df_dados_agrupados = pd.DataFrame(dados_agrupados)
    return df_dados_agrupados

# Supondo que 'lista_documentos_vetorizados' seja um DataFrame que já contém os vetores das sentenças,
# você pode chamar a função da seguinte maneira:
lista_documentos_agrupados = agrupar_dados_com_pos_tagging(lista_documentos_originais, lista_documentos_originais_pos, df_train, df_test)
lista_documentos_agrupados.head()
# A função assume que as variáveis 'lista_documentos_originais', 'lista_documentos_originais_pos', 'df_train' e 'df_test'
# já contêm os dados dos respectivos arquivos carregados.

# # Função para agrupar dados
# def agrupar_dados_com_pos_tagging(lista_documentos_originais, lista_documentos_originais_pos, df_train, df_test):
#     # Primeiro, criaremos um dicionário para mapear os IDs aos pós-tagging
#     pos_tagging_map = {row['id']: row['pos_documento'] for index, row in lista_documentos_originais_pos.iterrows()}

#     # Criaremos uma lista para armazenar os dados agrupados
#     dados_agrupados = []

#     # Agora, iremos iterar pelos dados de treino e teste para criar os pares de sentenças
#     for df in [df_train, df_test]:
#         for index, row in tqdm_notebook(df.iterrows(), desc='Agrupando dados', leave=False):
#             # Recuperamos o pós-tagging usando o ID das sentenças originais
#             pos_tagging1 = pos_tagging_map.get(row['#1 ID'])
#             pos_tagging2 = pos_tagging_map.get(row['#2 ID'])

#             # Adicionamos a linha formatada no novo esquema ao nosso conjunto de dados agrupados
#             dados_agrupados.append({
#                 "id": row['#1 ID'],
#                 "string1": pos_tagging1,
#                 "string2": pos_tagging2,
#                 "documento1": row['#1 String'],
#                 "documento2": row['#2 String'],
#                 "id_original1": row['#1 ID'],
#                 "id_original2": row['#2 ID'],
#                 "classes": row['Quality']
#             })

#     # Convertendo a lista de dados agrupados para um DataFrame pandas
#     df_dados_agrupados = pd.DataFrame(dados_agrupados)
#     return df_dados_agrupados

# # Agora vamos chamar a função e passar os datasets carregados
# lista_documentos_agrupados = agrupar_dados_com_pos_tagging(
#     lista_documentos_originais,
#     lista_documentos_originais_pos,
#     df_train,
#     df_test
# )

# Vamos exibir as primeiras linhas do DataFrame para verificar


,id,documento,pos_tagging,classe
0,702876_702977,"Amrozi accused his brother, whom he called the...","[[['Amrozi', 'accused', 'his', 'brother', ',',...",1
1,2108705_2108831,Yucaipa owned Dominick's before selling the ch...,"[[['Yucaipa', 'owned', 'Dominick', ""'s"", 'befo...",0
2,1330381_1330521,They had published an advertisement on the Int...,"[[['They', 'had', 'published', 'an', 'advertis...",1
3,3344667_3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","[[['Around', '0335', 'GMT', ',', 'Tab', 'share...",0
4,1236820_1236712,"The stock rose $2.11, or about 11 percent, to ...","[[['The', 'stock', 'rose', '$', '2.11', ',', '...",1


Converte em um dataframe

In [ ]:
lista_documentos_agrupados.documento[0]


'Amrozi accused his brother, whom he called the witness", of deliberately distorting his evidence. [SEP] Referring to him as only the witness", Amrozi accused his brother of deliberately distorting his evidence.'

In [ ]:
# Cria o dataframe da lista
# dflista_documentos_agrupados = pd.DataFrame(lista_documentos_agrupados, columns = ["id","sentencas","documento","classe"])
dflista_documentos_agrupados = pd.DataFrame(lista_documentos_agrupados, columns = ["id","documento","pos_tagging","classe"])

In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str,  "documento": str, "pos_tagging": object ,"classe": int}

dflista_documentos_agrupados = dflista_documentos_agrupados.astype(tipos)

In [ ]:
dflista_documentos_agrupados.sample(5)

,id,documento,pos_tagging,classe
2998,1798982_1799475,"She has a boyfriend, too, officials said: Sgt....","[[['She', 'has', 'a', 'boyfriend', ',', 'too',...",0
2522,869770_870219,PeopleSoft's board now has 10 business days to...,"[[['PeopleSoft', ""'s"", 'board', 'now', 'has', ...",1
5438,1448384_1448455,An incremental step reported by researchers at...,"[[['An', 'incremental', 'step', 'reported', 'b...",1
500,821523_821385,"Robert Liscouski, the Assistant Secretary of H...","[[['Robert', 'Liscouski', ',', 'the', 'Assista...",1
812,409061_408834,A base configuration with a 2.0GHz Intel Celer...,"[[['A', 'base', 'configuration', 'with', 'a', ...",1


In [ ]:
# Importa das bibliotecas
import pandas as pd

# Concatena as listas de documentos originais e perturbados
dflista_documentos_agrupados_pos = lista_documentos_originais_pos

logging.info("TERMINADO AGRUPAMENTO POS: {}.".format(len(dflista_documentos_agrupados_pos)))

INFO:root:TERMINADO AGRUPAMENTO POS: 10948.


In [ ]:
# Corrige os tipos dos dados da lista agrupada
tipos = {"id": str}

dflista_documentos_agrupados_pos = dflista_documentos_agrupados_pos.astype(tipos)

In [ ]:
dflista_documentos_agrupados_pos.sample(5)

,id,pos_documento
847,2579902,"[[[He, added, ,, There, 's, just, too, many, l..."
1715,144855,"[[[Judge, Harold, Baer, concluded, Wednesday, ..."
10014,1478869,"[[[Organizers, established, a, Web, site, ,, h..."
7917,1618799,"[[[Now, ,, with, the, agency, 's, last, three,..."
3286,2527646,"[[[Utah, 's, median, household, income, also, ..."


### 4.1.6 Renomeia o dataframe dos dados

In [ ]:
dfdados = dflista_documentos_agrupados

Apaga as colunas desnecessárias

In [ ]:
dfdados.sample(5)

,id,documento,pos_tagging,classe
3769,2654042_2653952,"Attorney Patricia Anderson, who represents the...","[[['Attorney', 'Patricia', 'Anderson', ',', 'w...",0
645,1588251_1588119,The technology-heavy Nasdaq Composite Index ga...,"[[['The', 'technology', '-', 'heavy', 'Nasdaq'...",0
5272,3368666_3368628,"Also Wednesday, a minibus detonated a roadside...","[[['Also', 'Wednesday', ',', 'a', 'minibus', '...",1
3436,937787_938238,Three no votes would kill it for now. [SEP] It...,"[[['Three', 'no', 'votes', 'would', 'kill', 'i...",1
1959,2630546_2630578,Wall Street was also waiting for aluminum make...,"[[['Wall', 'Street', 'was', 'also', 'waiting',...",1


## 4.2 Preparação dos dados

### Descartando os documentos muito grandes

Descarta os documentos que geram mais de 512 tokens com o tokenizador do BERT.

### Função descarte documentos muito grandes

In [ ]:
def descarteDocumentosGrandes(tamanho_maximo_token, dfdados):

  # Define o tamanho máximo para os tokens.
  tamanho_maximo = tamanho_maximo_token

  # Tokenize a codifica as setenças para o BERT.
  dfdados["input_ids"] = dfdados["documento"].apply(lambda tokens: tokenizer.encode(tokens, add_special_tokens=True))

  dfdados = dfdados[dfdados["input_ids"].apply(len)<tamanho_maximo]

  logging.info("Tamanho do conjunto de dados: {}.".format(len(dfdados)))

  # Remove as colunas desnecessárias.
  dfdados = dfdados.drop(columns=["input_ids"])

  # Informações do DataFrame
  print(dfdados.info())

  return dfdados

### Descartando documentos muito grandes

In [ ]:
dfdados = descarteDocumentosGrandes(model_args.max_seq_len, dfdados)

INFO:root:Tamanho do conjunto de dados: 5541.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5541 entries, 0 to 5540
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           5541 non-null   object
 1   documento    5541 non-null   object
 2   pos_tagging  5541 non-null   object
 3   classe       5541 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 173.3+ KB
None


### Procurando a maior quantidade de tokens no conjunto de dados

As células abaixo executará uma passagem de tokenização do conjunto de dados de treino, avaliação e teste para medir o comprimento máximo das documentos.

In [ ]:
maior_tamanho = 0

# Para cada documento no conjunto de treino.
for index, linha in dfdados.iterrows():

    # Tokeniza o texto e adiciona os tokens `[CLS]` e `[SEP]`.
    input_ids = tokenizer.encode(linha["documento"], add_special_tokens=True)

    # Atualiza o tamanho máximo de documento.
    maior_tamanho = max(maior_tamanho, len(input_ids))

logging.info("Tamanho máximo de documento no conjunto de dados: {}.".format(maior_tamanho))

INFO:root:Tamanho máximo de documento no conjunto de dados: 111.


In [ ]:
dfdados

,id,documento,pos_tagging,classe
0,702876_702977,"Amrozi accused his brother, whom he called the...","[[['Amrozi', 'accused', 'his', 'brother', ',',...",1
1,2108705_2108831,Yucaipa owned Dominick's before selling the ch...,"[[['Yucaipa', 'owned', 'Dominick', ""'s"", 'befo...",0
2,1330381_1330521,They had published an advertisement on the Int...,"[[['They', 'had', 'published', 'an', 'advertis...",1
3,3344667_3344648,"Around 0335 GMT, Tab shares were up 19 cents, ...","[[['Around', '0335', 'GMT', ',', 'Tab', 'share...",0
4,1236820_1236712,"The stock rose $2.11, or about 11 percent, to ...","[[['The', 'stock', 'rose', '$', '2.11', ',', '...",1
...,...,...,...,...
5536,2685984_2686122,"After Hughes refused to rehire Hernandez, he c...","[[['After', 'Hughes', 'refused', 'to', 'rehire...",0
5537,339215_339172,There are 103 Democrats in the Assembly and 47...,"[[['There', 'are', '103', 'Democrats', 'in', '...",0
5538,2996850_2996734,Bethany Hamilton remained in stable condition ...,"[[['Bethany', 'Hamilton', 'remained', 'in', 's...",0
5539,2095781_2095812,"Last week the power stations US owners, AES Co...","[[['Last', 'week', 'the', 'power', 'stations',...",1


## 4.3 Gerando os pares originais e permutados

Atributos da lista:
0. "id_original" - Nome do arquivo original
1. "documento_original" - Documento original
2. "classe_original" - Classe do documento original (1)
3. "id_permutado" - Nome do arquivo permutado
4. "documento_permutado" - Documento permutado
5. "classe_permutado" - Classe do documento permutado (0)

In [ ]:

lista = dfdados.values.tolist()

# Listas para documentos originais e permutados
lista_documentos_originais = []
lista_documentos_perturbados = []

# Para cada documento no conjunto de treino
for i in range(0, len(lista)):
    # Extrai as informações do documento atual
    id_documento, documento, pos_tagging, classe = lista[i]

    # Verifica se o documento é original ou perturbado baseado na classe
    if classe == 1:
        # Adiciona o documento à lista de originais
        lista_documentos_originais.append([id_documento, documento, pos_tagging, classe])
    else:
        # Adiciona o documento à lista de perturbados
        lista_documentos_perturbados.append([id_documento, documento, pos_tagging, classe])

# Aqui você terá duas listas separadas: uma para documentos originais e outra para perturbados
# Você pode processá-las conforme necessário a seguir

# Para converter cada lista em DataFrame e visualizar/manipular mais facilmente:
import pandas as pd

df_documentos_originais = pd.DataFrame(lista_documentos_originais, columns=['id_original', 'documento_original', 'pos_tagging_original', 'classe_original'])
df_documentos_perturbados = pd.DataFrame(lista_documentos_perturbados, columns=['id_perturbado', 'documento_perturbado', 'pos_tagging_perturbado', 'classe_perturbado'])

print("Documentos Originais:")
print(df_documentos_originais.head())

print("\nDocumentos Perturbados:")
print(df_documentos_perturbados.head())

# # Convertendo o DataFrame em uma lista de valores para facilitar a manipulação
# lista = dfdados.values.tolist()

# # Lista para armazenar os documentos agrupados
# lista_documentos = []

# # Itera sobre os elementos na lista de valores
# # Considerando que a lista contém todos os pares de documentos que você quer agrupar
# for i in range(0, len(lista), 2):  # Vai pular de dois em dois, já que estamos agrupando pares
#     # Garante que não tentaremos acessar um índice fora da lista
#     if i + 1 < len(lista):
#         # Cria um novo par agrupando informações dos dois documentos consecutivos
#         par_documentos = [
#             lista[i][0],      # id do primeiro documento
#             lista[i][1],      # documento1 do primeiro documento
#             lista[i+1][1],    # documento2 do segundo documento (supondo que este seja o par correspondente)
#             lista[i][3],      # id_original1 do primeiro documento
#             lista[i+1][3],    # id_original2 do segundo documento
#             lista[i][5],      # classe do primeiro documento
#         ]
#         # Adiciona o par criado à lista de documentos
#         lista_documentos.append(par_documentos)

# # Agora 'lista_documentos' contém todos os seus pares de documentos agrupados conforme necessário


Documentos Originais:
       id_original                                 documento_original  \
0    702876_702977  Amrozi accused his brother, whom he called the...   
1  1330381_1330521  They had published an advertisement on the Int...   
2  1236820_1236712  The stock rose $2.11, or about 11 percent, to ...   
3    738533_737951  Revenue in the first quarter of the year dropp...   
4    579975_579810  The DVD-CCA then appealed to the state Supreme...   

                                pos_tagging_original  classe_original  
0  [[['Amrozi', 'accused', 'his', 'brother', ',',...                1  
1  [[['They', 'had', 'published', 'an', 'advertis...                1  
2  [[['The', 'stock', 'rose', '$', '2.11', ',', '...                1  
3  [[['Revenue', 'in', 'the', 'first', 'quarter',...                1  
4  [[['The', 'DVD', '-', 'CCA', 'then', 'appealed...                1  

Documentos Perturbados:
     id_perturbado                               documento_perturbado  \
0  210870

### Transforma a lista em um dataframe

Atributos do dataframe:
0. "id_original" - Nome do arquivo original
1. "documento_original" - Documento original
2. "classe_original" - Classe do documento original (1)
3. "id_permutado" - Nome do arquivo permutado
4. "documento_permutado" - Documento permutado
5. "classe_permutado" - Classe do documento permutado (0)

In [ ]:
lista_documentos = dfdados.values.tolist()

In [ ]:
# Import das bibliotecas.
import pandas as pd

# Converte a lista em um dataframe.
dfdados = pd.DataFrame(lista_documentos, columns=[
                                                  "id",
                                                  "documento",
                                                  "pos_tagging",
                                                  "classe"
                                              ])

# Número de linhas carregadas do arquivo.
logging.info("Total de registros              : {}.".format(len(dfdados)))

INFO:root:Total de registros              : 5541.


Usaremos os pandas para analisar o conjunto de dados e examinar algumas de suas propriedades e pontos de dados.

In [ ]:
dfdados.sample(5)

,id,documento,pos_tagging,classe
1186,1805639_1805436,Printer maker Lexmark International Inc. spurt...,"[[['Printer', 'maker', 'Lexmark', 'Internation...",1
233,2206809_2206624,"She told Murray, We ... we have ... the fresh ...","[[['She', 'told', 'Murray', ',', 'We', '...'],...",1
1001,2636610_2636828,Sixty players and five coaches from the school...,"[[['Sixty', 'players', 'and', 'five', 'coaches...",0
2272,1276754_1276678,This is a cloud hanging over their credibility...,"[[['This', 'is', 'a', 'cloud', 'hanging', 'ove...",1
1984,2179199_2179139,Full classes of 48 are booked through Septembe...,"[[['Full', 'classes', 'of', '48', 'are', 'book...",1


## 4.3 Gerar divisão em k folds

### 4.3.1 Especifica os nomes dos arquivos de dados

In [ ]:
# Nome do arquivo
NOME_ARQUIVO_FOLD_COMPACTADO = DIRETORIO_COHEBERT + "_KFOLD_10" + "_P" +  str(model_args.documentos_perturbados) + "_K" + str(model_args.top_k_predicao) + ".zip"

### 4.3.2 Realizando a divisão do conjunto de dados para treinamento e validação através do k folds

In [ ]:
# Import das bibliotecas.
import numpy as np
from sklearn.model_selection import KFold

# Cria o diretório para receber os folds
# Diretório a ser criado
dirbase = "/content/validacao_kfold"

if not os.path.exists(dirbase):
    # Cria o diretório
    os.makedirs(dirbase)
    logging.info("Diretório criado: {}.".format(dirbase))
else:
    logging.info("Diretório já existe: {}.".format(dirbase))

# Parâmetros.
QTDE_FOLDS = model_args.fold

# Define o prefixo do nome dos arquivos dos folds
PREFIXO_NOME_ARQUIVO_TREINO = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Train_f"
PREFIXO_NOME_ARQUIVO_TESTE = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Test_f"

# Registra o tempo inícial.
t0 = time.time()

# Preparação do conjunto de dados.
X =  np.array(dfdados)

# Divisão em k folds(n_splits).
# shuffle, embaralha cada amostra.
# Quando shuffle igual True, random_state afeta a ordem dos índices,
# que controla a aleatoriedade de cada dobra.
kf = KFold(n_splits=QTDE_FOLDS,  random_state=True, shuffle=True)
print(kf)

TESTE = 1

# Percorre os indices do conjunto de dados.
for train_index, test_index in kf.split(X):
  logging.info("Executando divisão do fold: {}, Total: {}.".format(TESTE, len(train_index)+len(test_index)))
  logging.info("Treino: {}, Teste: {}.".format(len(train_index), len(test_index)))

  #print("Índices de treino:", len(train_index), " - ", train_index[0], " - ", train_index[len(train_index)-1])
  #print(train_index)
  #print("Índices de teste :", len(test_index), "  - ", test_index[0], " - ", test_index[len(test_index)-1])
  #print(test_index)

  # Recupera os dados das documentos para treino e teste.
  documentos_train = X[train_index]
  documentos_test  = X[test_index]

  # Organiza dados de treino.
  documentos_train_organizada = []

  # Adiciona a documento incoerente logo após a coerente para treino.
  for linha in documentos_train:
    # Documento original
    # linha[0] = "id_original" linha[1] = "documento_original" linha[2] = "classe_original"
    documentos_train_organizada.append((linha[0],linha[1],linha[3]))

  # Cria um dataframe com os dados de treinamento.
  pddata_tuples_train = pd.DataFrame(documentos_train_organizada, columns=["id","documento","classe"])

  # Salva o arquivo de treino do fold.
  pddata_tuples_train.to_csv(PREFIXO_NOME_ARQUIVO_TREINO + str(TESTE)+".csv", index = False, sep=';')

  # Organiza dados de teste.
  documentos_test_organizada = []

  # Adiciona a documento incoerente logo após a coerente para teste.
  for linha in documentos_test:
    # 1 Para coerente.
    # linha[0] = "id_original" linha[1] = "documento_original" linha[2] = "classe_original"
    documentos_test_organizada.append((linha[0],linha[1],linha[3]))

  # Cria um dataframe com os dados de teste.
  pddata_tuples_test = pd.DataFrame(documentos_test_organizada, columns=["id","documento","classe"])

  # Salva o arquivo de teste do fold.
  pddata_tuples_test.to_csv(PREFIXO_NOME_ARQUIVO_TESTE+str(TESTE)+".csv", index = False, sep=';')

  # Avança o contador de testes.
  TESTE = TESTE + 1

# Medida de quanto tempo levou a execução da validação.
teste_time = formataTempo(time.time() - t0)

logging.info("  Tempo gasto: {:}.".format(teste_time))

INFO:root:Diretório criado: /content/validacao_kfold.
INFO:root:Executando divisão do fold: 1, Total: 5541.
INFO:root:Treino: 4986, Teste: 555.
INFO:root:Executando divisão do fold: 2, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:Executando divisão do fold: 3, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:Executando divisão do fold: 4, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:Executando divisão do fold: 5, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:Executando divisão do fold: 6, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.


KFold(n_splits=10, random_state=True, shuffle=True)


INFO:root:Executando divisão do fold: 7, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:Executando divisão do fold: 8, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:Executando divisão do fold: 9, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:Executando divisão do fold: 10, Total: 5541.
INFO:root:Treino: 4987, Teste: 554.
INFO:root:  Tempo gasto: 0:00:00.


### 4.3.3 Analisando os dados gerados na divisão

In [ ]:
# Lista para armazenar os dados
lista_dados_train_folds = []
lista_dados_test_folds = []

# Define o prefixo do nome dos arquivos dos folds
PREFIXO_NOME_ARQUIVO_TREINO = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Train_f"
PREFIXO_NOME_ARQUIVO_TESTE = "/content/validacao_kfold/" + DIRETORIO_COHEBERT + "_Test_f"

for x in range(QTDE_FOLDS):

  dadostrain = pd.read_csv(PREFIXO_NOME_ARQUIVO_TREINO+str(x+1)+".csv", sep=';')
  logging.info("Dados treino do fold {}: {}.".format(x+1,len(dadostrain)))

  dadostest = pd.read_csv(PREFIXO_NOME_ARQUIVO_TESTE+str(x+1)+".csv", sep=';')
  logging.info("Dados teste do fold {}: {}.".format(x+1,len(dadostest)))

  lista_dados_train_folds.append([x,dadostrain.classe.sum(), len(dadostrain.classe)-dadostrain.classe.sum()])
  lista_dados_test_folds.append([x,dadostest.classe.sum(), len(dadostest.classe)-dadostest.classe.sum()])

  # Pega as listas de documentos e seus rótulos.
  documentos = dadostrain.documento.values
  maior_tamanho_documento_treino = 0
  # Para cada documento no conjunto de treino.
  for documento in documentos:
      # Tokeniza o texto e adiciona os tokens `[CLS]` e `[SEP]`.
      input_ids = tokenizer.encode(documento, add_special_tokens=True)
      # Atualiza o tamanho máximo de documento.
      maior_tamanho_documento_treino = max(maior_tamanho_documento_treino, len(input_ids))

  logging.info("Máximo de tokens no conjunto de dados de treino: {}.".format(maior_tamanho_documento_treino))

  # Pega as listas de documentos e seus rótulos.
  documentos = dadostest.documento.values
  maior_tamanho_documento_teste = 0
  # Para cada documento no conjunto de treino.
  for documento in documentos:
      # Tokeniza o texto e adiciona os tokens `[CLS]` e `[SEP]`.
      input_ids = tokenizer.encode(documento, add_special_tokens=True)
      # Atualiza o tamanho máximo de documento.
      maior_tamanho_documento_teste = max(maior_tamanho_documento_teste, len(input_ids))

  logging.info("Máximo de token no conjunto de dados de teste: {}.".format(maior_tamanho_documento_teste))

  logging.info("Fold {} Treino positivos: {} de {} ({:.2f}%)".format(x+1,
                                                             dadostrain.classe.sum(),
                                                             len(dadostrain.classe),
                                                             (dadostrain.classe.sum() / len(dadostrain.classe) * 100.0)
                                                             ))

  logging.info("Fold {} Treino negativos: {} de {} ({:.2f}%)".format(x+1,
                                                             len(dadostrain.classe)-dadostrain.classe.sum(),
                                                             len(dadostrain.classe),
                                                             ((len(dadostrain.classe)-dadostrain.classe.sum()) / len(dadostrain.classe) * 100.0)))

  logging.info("Fold {} Teste positivos: {} de {} ({:.2f}%)".format(x+1,
                                                             dadostest.classe.sum(),
                                                             len(dadostest.classe),
                                                             (dadostest.classe.sum() / len(dadostest.classe) * 100.0)))
  logging.info("Fold {} Teste negativos: {} de {} ({:.2f}%)".format(x+1,
                                                             len(dadostest.classe)-dadostest.classe.sum(),
                                                             len(dadostest.classe),
                                                             ((len(dadostest.classe)-dadostest.classe.sum()) / len(dadostest.classe) * 100.0)))
  logging.info("")


INFO:root:Dados treino do fold 1: 4986.
INFO:root:Dados teste do fold 1: 555.
INFO:root:Máximo de tokens no conjunto de dados de treino: 106.
INFO:root:Máximo de token no conjunto de dados de teste: 111.
INFO:root:Fold 1 Treino positivos: 3350 de 4986 (67.19%)
INFO:root:Fold 1 Treino negativos: 1636 de 4986 (32.81%)
INFO:root:Fold 1 Teste positivos: 381 de 555 (68.65%)
INFO:root:Fold 1 Teste negativos: 174 de 555 (31.35%)
INFO:root:
INFO:root:Dados treino do fold 2: 4987.
INFO:root:Dados teste do fold 2: 554.
INFO:root:Máximo de tokens no conjunto de dados de treino: 111.
INFO:root:Máximo de token no conjunto de dados de teste: 99.
INFO:root:Fold 2 Treino positivos: 3370 de 4987 (67.58%)
INFO:root:Fold 2 Treino negativos: 1617 de 4987 (32.42%)
INFO:root:Fold 2 Teste positivos: 361 de 554 (65.16%)
INFO:root:Fold 2 Teste negativos: 193 de 554 (34.84%)
INFO:root:
INFO:root:Dados treino do fold 3: 4987.
INFO:root:Dados teste do fold 3: 554.
INFO:root:Máximo de tokens no conjunto de dados d

### 4.3.4 Compacta os arquivos dos folds

Compacta os arquivos em dos folds

A opção `-jr` do zip compacta os arquivos ignorando a estrutura de diretório.

In [ ]:
!zip -j -r "validacao_kfold/""$NOME_ARQUIVO_FOLD_COMPACTADO" "validacao_kfold/""$DIRETORIO_COHEBERT"_*.csv

logging.info("Terminei a compactação.")

  adding: cohquad_Test_f10.csv (deflated 64%)
  adding: cohquad_Test_f1.csv (deflated 64%)
  adding: cohquad_Test_f2.csv (deflated 64%)
  adding: cohquad_Test_f3.csv (deflated 64%)
  adding: cohquad_Test_f4.csv (deflated 64%)
  adding: cohquad_Test_f5.csv (deflated 64%)
  adding: cohquad_Test_f6.csv (deflated 64%)
  adding: cohquad_Test_f7.csv (deflated 64%)
  adding: cohquad_Test_f8.csv (deflated 64%)
  adding: cohquad_Test_f9.csv (deflated 64%)
  adding: cohquad_Train_f10.csv (deflated 65%)
  adding: cohquad_Train_f1.csv (deflated 65%)
  adding: cohquad_Train_f2.csv (deflated 65%)
  adding: cohquad_Train_f3.csv (deflated 65%)
  adding: cohquad_Train_f4.csv (deflated 65%)
  adding: cohquad_Train_f5.csv (deflated 65%)
  adding: cohquad_Train_f6.csv (deflated 65%)
  adding: cohquad_Train_f7.csv (deflated 65%)
  adding: cohquad_Train_f8.csv (deflated 65%)
  adding: cohquad_Train_f9.csv (deflated 65%)


INFO:root:Terminei a compactação.


Copia o arquivo compactado para o google drive

In [ ]:
# !cp "validacao_kfold/""$NOME_ARQUIVO_FOLD_COMPACTADO" "/content/drive/My Drive/Colab Notebooks/Data/""$DIRETORIO_COHEBERT""/validacao_classificacao_palavra/kfold"

# logging.info("Terminei a cópia.")

# Defina o diretório e nome do arquivo compactado
DIRETORIO_VALIDACAO = 'validacao_kfold/'

# Caminho completo para o arquivo compactado
caminho_arquivo_compactado = os.path.join(DIRETORIO_VALIDACAO, NOME_ARQUIVO_FOLD_COMPACTADO)

destination_blob_name = f'{DIRETORIO_COHEBERT}/validacao_classificacao_palavra/kfold/{NOME_ARQUIVO_FOLD_COMPACTADO}'
upload_blob(bucket_name, caminho_arquivo_compactado, destination_blob_name)

logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


File validacao_kfold/cohquad_KFOLD_10_P1_K1.zip uploaded to cohquad/validacao_classificacao_palavra/kfold/cohquad_KFOLD_10_P1_K1.zip.


# 5 Finalização

## 5.1 Tempo final de processamento



In [ ]:
 # Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  1:06:21 (h:mm:ss)
